<a href="https://colab.research.google.com/github/tiagopecurto/tiagopecurto/blob/main/docs/labs/projs/spbp2425_tp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistemas para Processamento de Big Data
## TP1 - Energy Meter Monitoring




The sensor data corresponds to regular readings from 11 residential energy meters. The data covers the month of February 2024.

Each data sample has the following schema:

timestamp | sensor_id | energy
----------|-------------|-----------
timestamp | string  | float

Each energy value (KWh) corresponds to the accumulated value of the meter at the time of measurement. As such,
each meter is expected to produce a monotonically increasing series of pairs of timestamp and energy consummed up to that moment.

The meters do not start at zero or at the same value.


## Questions

The following questions should be answered for the month of February and only for this month.

### For the group of sensors:

1. Compute the total energy consumed.

2. Compute the running total energy consumed so far for each day, inclusive.

Note: You can approximate the result but using the last reading of each day from each sensor.

### For each sensor, separately:

3. Compute the total energy consumed and the average energy consumption per day.

4. Compute the day of the month with minimum and maximum energy consumption.

Note: You can approximate the result but using the last reading of each day from each sensor.

### For each sensor, separately, with estimations:

**Assumptions:**

+ Readings may be missing for extended periods due to communication problems with the sensors.

+ Readings are collected do not fall precisely "on the hour". The are collected and recorded any time.

+ For more precise results, estimate the value of the meter at precise timestamp, using linear interpolation from nearest readings.

5. Compute the **estimated** value of each sensor meter for every hour and day of the month (in ascending order).

6. Compute the **estimated** running total of the energy consumed so far. The value should be updated every hour.

## Requeriments

Solve each question using Structured Spark, either Dataframes or SQL or both.

## Other Grading Criteria

+ Grading will also take into account the general clarity of the programming and of the presentation report (notebook).




### Deadline
+ November 10, 23h59

For each day late, ***0.5 / day penalty***. Penalty accumulates until the grade of the assignment reaches 8.0.

---
### Colab Setup


In [ ]:
#@title Install PySpark
!pip install pyspark findspark --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
#@title Download the dataset

!wget -q -O energy-readings.csv https://raw.githubusercontent.com/smduarte/spbd-2425/refs/heads/main/docs/labs/projs/energy-readings.csv
!head -2 energy-readings.csv

date;sensor;energy
2024-02-01 00:00:00;D;2615.0


In [12]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]') \
						.appName('energy').getOrCreate()

sc = spark.sparkContext
try :
    readings = spark.read.csv('energy-readings.csv',
                             sep =';', header=True, inferSchema=True)

    # Filtrar apenas para o mês de Fevereiro
    february_data = readings.filter(col("date").substr(6,2) == 2)
    #february_data.show()

    # Calcular o consumo total de energia para cada sensor e ordenado de forma ascendente
    sensor_total_energy = february_data.groupBy("sensor") \
    .agg((max("energy") - min("energy")).alias("total_energy")) \
    .orderBy("sensor", ascending=True)

    sensor_total_energy.show()

    #Calcular o Total Acumulado Diário

    daily_max_energy = february_data.withColumn("day", date_trunc("day", "date")) \
    .groupBy("sensor", "day") \
    .agg(max("energy").alias("daily_energy"))





    #readings.printSchema()
    #readings.show(5)
except Exception as err:
    print(err)

+------+------------------+
|sensor|      total_energy|
+------+------------------+
|     A|            166.38|
|     B|129.80999999999995|
|     C|            257.22|
|     D| 487.4000000000001|
|     E| 448.7600000000002|
|     F|160.40999999999997|
|     G| 168.4699999999999|
|     H| 479.5999999999999|
|     I|351.40999999999985|
|     J| 230.8499999999999|
|     K|             226.5|
+------+------------------+

